Augmentation


Tokenization

Stemming and Lemmatization

Removing stopwords, punctuation, HTML tags, emojis

Vectorization: Bag of Words (BoW), Term Frequency - Inverse Document Frequency (TF-IDF), Word2Vec, GloVe

Naive Bayes, SVM, Random Forest algorithm testing